<a href="https://colab.research.google.com/github/YuBaichuan2000/Customer_Churn_ANN/blob/main/Digit_Recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

In [6]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [7]:
df = pd.read_csv('train.csv')
df.shape

(42000, 785)

## Create X and y for model

In [8]:
X = df.drop('label',axis='columns')
y = df['label']

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=5)

## Data Scaling

In [10]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

## Reshape Dataframe to use CNN

In [11]:
X_train_scaled = X_train_scaled.to_numpy()
X_test_scaled = X_test_scaled.to_numpy()

In [12]:
X_train_3d = X_train_scaled.reshape(X_train_scaled.shape[0],28,28,1)
X_test_3d = X_test_scaled.reshape(X_test_scaled.shape[0],28,28,1)


In [14]:
model = keras.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation="relu", input_shape=(28,28,1)), # 32 filters to detect 32 sections, 3 by 3 filter
    keras.layers.MaxPooling2D((2,2)), # 2 by 2 max pooling
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation="relu", input_shape=(28,28,1)), # 64 filters to detect 64 sections, 3 by 3 filter
    keras.layers.MaxPooling2D((2,2)), # 2 by 2 max pooling

    keras.layers.Flatten(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(10, activation="sigmoid")
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])
model.fit(X_train_3d, y_train, epochs=20)

Epoch 1/20
1050/1050 [==============================] - 38s 35ms/step - loss: 0.1940 - accuracy: 0.9378
Epoch 2/20
1050/1050 [==============================] - 36s 35ms/step - loss: 0.0584 - accuracy: 0.9813
Epoch 3/20
1050/1050 [==============================] - 38s 36ms/step - loss: 0.0400 - accuracy: 0.9872
Epoch 4/20
1050/1050 [==============================] - 33s 32ms/step - loss: 0.0304 - accuracy: 0.9903
Epoch 5/20
1050/1050 [==============================] - 36s 34ms/step - loss: 0.0226 - accuracy: 0.9926
Epoch 6/20
1050/1050 [==============================] - 36s 34ms/step - loss: 0.0174 - accuracy: 0.9939
Epoch 7/20
1050/1050 [==============================] - 34s 32ms/step - loss: 0.0166 - accuracy: 0.9945
Epoch 8/20
1050/1050 [==============================] - 35s 34ms/step - loss: 0.0137 - accuracy: 0.9949
Epoch 9/20
1050/1050 [==============================] - 36s 34ms/step - loss: 0.0112 - accuracy: 0.9961
Epoch 10/20
1050/1050 [==============================] - 34s 32m

In [15]:
model.evaluate(X_test_3d, y_test)

263/263 [==============================] - 3s 12ms/step - loss: 0.0632 - accuracy: 0.9889


[0.06317424029111862, 0.9889285564422607]

## Testing

In [16]:
df_test = pd.read_csv('test.csv')

In [18]:
df_test = df_test / 255
df_test = df_test.to_numpy()
df_test_3d = df_test.reshape(df_test.shape[0],28,28,1)
df_test_3d.shape

(28000, 28, 28, 1)

In [20]:
y_pred = model.predict(df_test_3d)

875/875 [==============================] - 10s 11ms/step


In [28]:
y_pred_labels = [np.argmax(i) for i in y_pred]

[2, 0, 9, 9, 3]

In [45]:
df_result = pd.DataFrame(y_pred_labels, columns = ['Label'])

In [46]:
df_result['ImageId'] = np.arange(1,28001)

In [51]:
df_result = df_result.reindex(columns=['ImageId', 'Label'])
df_result.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [53]:
df_result.to_csv('submission.csv', index=False)
